In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    
    chunk_overlap=200, 
)

loader = Docx2txtLoader("./tax.docx")
document_lists = loader.load_and_split(text_splitter=text_splitter)
len(document_lists)

183

In [ ]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [8]:
from langchain_chroma import Chroma

database = Chroma(collection_name='chroma_tax', persist_directory='./chroma_db', embedding_function=embeddings)

In [10]:
query = '연봉이 5000만원인 직장인은 소득세로 얼마를 내야하나요?'
# 컨텍스트 길이 제한으로 인해 검색 문서 수를 줄임
returied_docs = database.similarity_search(query)

InvalidArgumentError: Collection expecting embedding with dimension of 3072, got 4096

In [ ]:
returied_docs

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(base_url="http://localhost:8000/v1", api_key="dummy", model="yanolja/EEVE-Korean-Instruct-2.8B-v1.0", temperature=0.7)
llm = ChatOpenAI()

In [ ]:
prompt = f"""[identity]
- 당신은 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
- 소득세 관련 법령: {returied_docs}

Question: {query}
"""

In [ ]:
ai_msg = llm.invoke(prompt)

In [ ]:
ai_msg.content

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [ ]:
from langchain.chains import create_retrieval_chain

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
ai_msg = qa_chain({"query": query})

In [ ]:
ai_msg